In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [ ]:
# to reload
# %dotenv

# Use ETrade API

In [2]:
import pandas as pd
from datetime import date, datetime, timedelta

import os
import sys
sys.path.append("../projects/gain_tracker")

In [3]:
from gain_tracker.resources.etrade_api import ETradeAPI
from gain_tracker.assets.positions import camel_to_snake

/workspaces/src/trader_buddy/jupyter-notebooks/../projects/gain_tracker/gain_tracker/assets/positions.py:90: ExperimentalWarning: Parameter `allow_nonexistent_upstream_partitions` of initializer `TimeWindowPartitionMapping.__init__` is experimental. It may break in future versions, even between dot releases. To mute warnings for experimental functionality, invoke warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning) or use one of the other methods described at https://docs.python.org/3/library/warnings.html#describing-warning-filters.
  partition_mapping=TimeWindowPartitionMapping(


In [4]:
# consumer_key = os.getenv("ETRADE_SANDBOX_KEY")
# consumer_secret = os.getenv("ETRADE_SANDBOX_SECRET")
consumer_key = os.getenv("ETRADE_KEY")
consumer_secret = os.getenv("ETRADE_SECRET")

In [5]:
etrader = ETradeAPI("prod")

In [6]:
etrader.authenticate_session()

Please accept agreement and enter verification code from browser:  XEOYM


print(etrader.dagster_config_input_string())

In [40]:
accounts = etrader.list_accounts()

In [49]:
acc_date = date(2024, 2, 4)

In [48]:
accounts_df = pd.DataFrame(accounts)

snake_cols = {c:camel_to_snake(c) for c in accounts_df.columns}
accounts_df.rename(columns=snake_cols, inplace=True)

In [61]:
acc_dfs = []

while acc_date < date(2024, 3, 25):
    accounts_df.loc[:, 'date'] = acc_date
    acc_dfs.append(accounts_df.copy(deep=True)) 
    acc_date = acc_date + timedelta(days=1)

In [62]:
acc_backfill = pd.concat(acc_dfs)

In [56]:
from google.cloud import bigquery

client = bigquery.Client()

In [63]:
job = client.load_table_from_dataframe(
    acc_backfill.set_index('account_id'), "main-street-labs.gain_tracker.etrade_accounts"
)

In [30]:
a_etrader = ETradeAPI("prod", session_token=etrader.session_token, session_token_secret=etrader.session_token_secret)

print(a_etrader.dagster_config_input_string())

In [31]:
session = a_etrader.create_authenticated_session()

In [11]:
response = session.get("https://api.etrade.com/oauth/renew_access_token")
response.status_code

200

In [38]:
# list_accounts_url = 'https://apisb.etrade.com/v1/accounts/list.json'
list_accounts_url = 'https://api.etrade.com/v1/accounts/list.json'

In [39]:
resp = session.get(list_accounts_url, params = {'format': 'json'})

accounts = pd.DataFrame(resp.json()["AccountListResponse"]["Accounts"]["Account"])

accounts

,accountId,accountIdKey,accountMode,accountDesc,accountName,accountType,institutionType,accountStatus,closedDate,shareWorksAccount,fcManagedMssbClosedAccount
0,153285977,lP_HmzzVyiLFftb7DXaEUw,CASH,Individual Brokerage,,INDIVIDUAL,BROKERAGE,ACTIVE,0,False,False
1,227699084,E2ayfoKSzH1fFZs-7xV48A,CASH,Roth IRA,,ROTHIRA,BROKERAGE,ACTIVE,0,False,False
2,892405345,SXPtmpM7IZIpymUjhjI-zg,CASH,Traditional IRA,,CONTRIBUTORY,BROKERAGE,ACTIVE,0,False,False


In [32]:
accountIdKey = "SXPtmpM7IZIpymUjhjI-zg"
accountIdKey = "E2ayfoKSzH1fFZs-7xV48A"
instType = "BROKERAGE"

In [36]:
transactions = a_etrader.get_transactions(accountIdKey, date_range=("02042024", "02112024"))
# transactions = a_etrader.get_transactions(accountIdKey, start_end_dates=("03112024", "03182024"))

len(transactions)

{'startDate': '02042024', 'endDate': '02112024'}


5

In [34]:
transactions.columns

Index(['transactionId', 'accountId', 'transactionDate', 'postDate', 'amount',
       'description', 'transactionType', 'memo', 'imageFlag', 'instType',
       'storeId', 'detailsURI', 'quantity', 'price', 'settlementCurrency',
       'paymentCurrency', 'fee', 'displaySymbol', 'settlementDate', 'symbol',
       'securityType'],
      dtype='object')

In [35]:
transactions.tail(3)

,transactionId,accountId,transactionDate,postDate,amount,description,transactionType,memo,imageFlag,instType,...,detailsURI,quantity,price,settlementCurrency,paymentCurrency,fee,displaySymbol,settlementDate,symbol,securityType
32,24228501183016,227699084,1723739401000,1723791600000,460.30,TETRA TECH INC UNSOLICITED TRADE,Sold,,False,BROKERAGE,...,https://api.etrade.com/v1/accounts/E2ayfoKSzH1...,-2.0,230.16,USD,USD,0.02,TTEK,1723791600000,TTEK,EQ
33,24228501183015,227699084,1723739400000,1723791600000,583.38,SUPER MICRO COMPUTER INC UNSOLICITED TRADE,Sold,,False,BROKERAGE,...,https://api.etrade.com/v1/accounts/E2ayfoKSzH1...,-1.0,583.40,USD,USD,0.02,SMCI,1723791600000,SMCI,EQ
34,24227500383170,227699084,1723676095000,1723705200000,-193.00,ITRON INC UNSOLICITED TRADE,Bought,,False,BROKERAGE,...,https://api.etrade.com/v1/accounts/E2ayfoKSzH1...,2.0,96.50,USD,USD,0.00,ITRI,1723705200000,ITRI,EQ


In [96]:
transactions.columns

Index(['transactionId', 'accountId', 'transactionDate', 'postDate', 'amount',
       'description', 'transactionType', 'memo', 'imageFlag', 'instType',
       'storeId', 'brokerage', 'detailsURI', 'product', 'quantity', 'price',
       'settlementCurrency', 'paymentCurrency', 'fee', 'displaySymbol',
       'settlementDate'],
      dtype='object')

In [51]:
transactions_url = f"https://api.etrade.com/v1/accounts/{accountIdKey}/transactions.json"

In [61]:
t_resp = session.get(transactions_url, params={"startDate": "03112024", "endDate": "03192024"})

In [62]:
tdata = t_resp.json()

In [63]:
tdata["TransactionListResponse"].keys()

dict_keys(['pageMarkers', 'moreTransactions', 'transactionCount', 'totalCount', 'Transaction'])

In [64]:
transactions = pd.DataFrame(tdata["TransactionListResponse"]["Transaction"])
len(transactions)

11

In [74]:
sold = transactions.loc[transactions["transactionType"]=="Sold"].set_index(["displaySymbol", "amount"])

In [76]:
sold.loc[("RIVN", 231.19)]

transactionId                                            24067500657444
accountId                                                     227699084
transactionDate                                           1709832601000
postDate                                                  1710140400000
description                RIVIAN AUTOMOTIVE INC CL A UNSOLICITED TRADE
transactionType                                                    Sold
memo                                                                   
imageFlag                                                         False
instType                                                      BROKERAGE
storeId                                                               0
brokerage             {'product': {'symbol': 'RIVN', 'securityType':...
detailsURI            https://api.etrade.com/v1/accounts/E2ayfoKSzH1...
product                        {'symbol': 'RIVN', 'securityType': 'EQ'}
quantity                                                        

In [72]:
transactions.loc[transactions["transactionType"]=="Sold"].head(3)

,transactionId,accountId,transactionDate,postDate,amount,description,transactionType,memo,imageFlag,instType,...,brokerage,detailsURI,product,quantity,price,settlementCurrency,paymentCurrency,fee,displaySymbol,settlementDate
13,24067500657444,227699084,1709832601000,1710140400000,231.19,RIVIAN AUTOMOTIVE INC CL A UNSOLICITED TRADE,Sold,,False,BROKERAGE,...,"{'product': {'symbol': 'RIVN', 'securityType':...",https://api.etrade.com/v1/accounts/E2ayfoKSzH1...,"{'symbol': 'RIVN', 'securityType': 'EQ'}",-20.0,11.56,USD,USD,0.01,RIVN,1710140400000
16,24066500771634,227699084,1709746285000,1709884800000,389.99,HANNON ARMSTRONG SUS INFRA CAP UNSOLICITED TRADE,Sold,,False,BROKERAGE,...,"{'product': {'symbol': 'HASI', 'securityType':...",https://api.etrade.com/v1/accounts/E2ayfoKSzH1...,"{'symbol': 'HASI', 'securityType': 'EQ'}",-15.0,26.00,USD,USD,0.01,HASI,1709884800000
19,24064500884447,227699084,1709580755000,1709712000000,1085.99,TETRA TECH INC UNSOLICITED TRADE,Sold,,False,BROKERAGE,...,"{'product': {'symbol': 'TTEK', 'securityType':...",https://api.etrade.com/v1/accounts/E2ayfoKSzH1...,"{'symbol': 'TTEK', 'securityType': 'EQ'}",-6.0,181.00,USD,USD,0.01,TTEK,1709712000000


In [73]:
transactions.columns

Index(['transactionId', 'accountId', 'transactionDate', 'postDate', 'amount',
       'description', 'transactionType', 'memo', 'imageFlag', 'instType',
       'storeId', 'brokerage', 'detailsURI', 'product', 'quantity', 'price',
       'settlementCurrency', 'paymentCurrency', 'fee', 'displaySymbol',
       'settlementDate'],
      dtype='object')

In [20]:
transactions.loc[transactions["transactionType"]=="Sold"].loc[30]["brokerage"]

{'product': {'symbol': 'WMT', 'securityType': 'EQ'},
 'quantity': -15,
 'price': 59.8,
 'settlementCurrency': 'USD',
 'paymentCurrency': 'USD',
 'fee': 0.01,
 'displaySymbol': 'WMT',
 'settlementDate': 1709107200000}

In [101]:
transactions["transactionType"].unique()

array(['Bought', 'Sold', 'Interest', 'Reorganization', 'Dividend'],
      dtype=object)

### for transactions details, not used at this time

In [109]:
session.headers["Accept"] = "application/json"

In [110]:
session.headers

{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/json', 'Connection': 'keep-alive'}

In [118]:
td_resp = session.get('https://api.etrade.com/v1/accounts/E2ayfoKSzH1fFZs-7xV48A/transactions/24047500707477')

In [119]:
td_resp.json()

{'TransactionDetailsResponse': {'transactionId': 24047500707477,
  'accountId': '227699084',
  'transactionDate': 1708113439000,
  'amount': 1079.99,
  'description': 'CONS DISCRET SEL SECT SPDR FD UNSOLICITED TRADE',
  'Category': {'categoryId': '0',
   'parentId': '0',
   'categoryName': '',
   'parentName': ''},
  'Brokerage': {'transactionType': 'Sold',
   'quantity': -6.0,
   'price': 180.0,
   'settlementCurrency': 'USD',
   'paymentCurrency': 'USD',
   'fee': 0.01,
   'memo': '',
   'orderNo': '206',
   'Product': {'symbol': 'XLY'}}}}

In [39]:
account_balances_url = f"https://apisb.etrade.com/v1/accounts/{accountIdKey}/balance.json"

In [75]:
account_balances_url

'https://apisb.etrade.com/v1/accounts/xj1Dc18FTqWPqkEEVUr5rw/balance'

In [76]:
resp = session.get(account_balances_url, params = {"instType":instType,"realNAV": False})

In [77]:
resp.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><BalanceResponse><accountId>833535350</accountId><accountType>MARGIN</accountType><optionLevel>LEVEL_4</optionLevel><accountDescription></accountDescription><Cash><fundsForOpenOrdersCash>0.0</fundsForOpenOrdersCash><moneyMktBalance>0.0</moneyMktBalance></Cash><Computed><cashAvailableForInvestment>0.00</cashAvailableForInvestment><cashAvailableForWithdrawal>0.00</cashAvailableForWithdrawal><netCash>-740.450013</netCash><cashBalance>0.0</cashBalance><settledCashForInvestment>0.0</settledCashForInvestment><unSettledCashForInvestment>0.0</unSettledCashForInvestment><fundsWithheldFromPurchasePower>0.0</fundsWithheldFromPurchasePower><fundsWithheldFromWithdrawal>0.0</fundsWithheldFromWithdrawal><OpenCalls><cashCall>0.0</cashCall></OpenCalls><RealTimeValues><totalAccountValue>0.0</totalAccountValue><netMv>-454.22</netMv><netMvLong>0.0</netMvLong></RealTimeValues></Computed></BalanceResponse>'

In [101]:
positions = a_etrader.view_portfolio(accountIdKey)

In [102]:
positions[["symbolDescription", "pricePaid", "dateAcquired", "positionLotId", "positionId"]].head()

,symbolDescription,pricePaid,dateAcquired,positionLotId,positionId
0,USMV,43.640,1458532800000,9965198006,10227202005
1,IVV,210.130,1461643200000,15384288006,14848902005
2,IVV,495.500,1707800400000,2343150552103,14848902005
3,ITOT,47.105,1462161600000,16438625006,22411007005
4,ITOT,74.350,1596513600000,749658972103,22411007005


In [105]:
positions.set_index("symbolDescription").loc["USMV", "lotsDetails"]

'https://api.etrade.com/v1/accounts/E2ayfoKSzH1fFZs-7xV48A/portfolio/10227202005.json'

In [40]:
# view_portfolio_url = f"https://apisb.etrade.com/v1/accounts/{accountIdKey}/portfolio.json"
view_portfolio_url = f"https://api.etrade.com/v1/accounts/{accountIdKey}/portfolio.json"

In [57]:
viewp_resp = session.get(view_portfolio_url, params={"view": "QUICK", "lotsRequired": True})

In [58]:
data = viewp_resp.json()

In [59]:
ps_x = data["PortfolioResponse"]["AccountPortfolio"][0]["Position"]

In [61]:
data["PortfolioResponse"]["AccountPortfolio"][0]

{'accountId': '892405345',
 'Position': [{'positionId': 356478010900,
   'symbolDescription': 'AIRR',
   'dateAcquired': 1704430800000,
   'pricePaid': 55.1,
   'commissions': 0,
   'otherFees': 0,
   'quantity': 10,
   'positionIndicator': 'TYPE1',
   'positionType': 'LONG',
   'daysGain': 1.3,
   'daysGainPct': 0.2042,
   'marketValue': 637.7999,
   'totalCost': 551,
   'totalGain': 86.7999,
   'totalGainPct': 15.7531,
   'pctOfPortfolio': 4.6407,
   'costPerShare': 55.1,
   'todayCommissions': 0,
   'todayFees': 0,
   'todayPricePaid': 0,
   'todayQuantity': 0,
   'adjPrevClose': 63.65,
   'lotsDetails': 'https://api.etrade.com/v1/accounts/SXPtmpM7IZIpymUjhjI-zg/portfolio/356478010900.json',
   'quoteDetails': 'https://api.etrade.com/v1/market/quote/AIRR.json',
   'Product': {'symbol': 'AIRR',
    'securityType': 'EQ',
    'securitySubType': 'ETF',
    'expiryYear': 0,
    'expiryMonth': 0,
    'expiryDay': 0,
    'strikePrice': 0,
    'productId': {'symbol': 'AIRR', 'typeCode': 'EQ

In [110]:
l_resp = session.get("https://api.etrade.com/v1/accounts/SXPtmpM7IZIpymUjhjI-zg/portfolio/364189645900.json")
# l_resp = session.get('https://api.etrade.com/v1/accounts/E2ayfoKSzH1fFZs-7xV48A/portfolio/10227202005.json')

In [111]:
lot_data = l_resp.json()

In [112]:
lot_data

{'PositionLotsResponse': {'shortType': 1,
  'PositionLot': [{'positionId': 364189645900,
    'positionLotId': 2347253384103,
    'price': 475.0,
    'termCode': 2,
    'daysGain': -34.0,
    'daysGainPct': -7.7448,
    'marketValue': 405.0,
    'totalCost': 475.0,
    'totalCostForGainPct': -14.7368,
    'totalGain': -70.0,
    'lotSourceCode': 1,
    'originalQty': 1,
    'remainingQty': 1,
    'availableQty': 0,
    'orderNo': 99,
    'legNo': 1,
    'acquiredDate': 1707973200000,
    'locationCode': 1,
    'exchangeRate': 1.0,
    'settlementCurrency': 'USD',
    'paymentCurrency': 'USD',
    'adjPrice': 0.0,
    'commPerShare': 0.0,
    'feesPerShare': 0.0,
    'shortType': 1},
   {'positionId': 364189645900,
    'positionLotId': 2357680677103,
    'price': 444.0,
    'termCode': 2,
    'daysGain': -34.0,
    'daysGainPct': -7.7448,
    'marketValue': 405.0,
    'totalCost': 444.0,
    'totalCostForGainPct': -8.7837,
    'totalGain': -39.0,
    'lotSourceCode': 1,
    'originalQty'

In [60]:
[p for p in ps_x if p["symbolDescription"] == "MDB"]

[{'positionId': 364189645900,
  'symbolDescription': 'MDB',
  'dateAcquired': 1707973200000,
  'pricePaid': 459.5,
  'commissions': 0,
  'otherFees': 0,
  'quantity': 2,
  'positionIndicator': 'TYPE1',
  'positionType': 'LONG',
  'daysGain': -7.16,
  'daysGainPct': -0.7998,
  'marketValue': 888,
  'totalCost': 919,
  'totalGain': -31,
  'totalGainPct': -3.3732,
  'pctOfPortfolio': 6.4612,
  'costPerShare': 459.5,
  'todayCommissions': 0,
  'todayFees': 0,
  'todayPricePaid': 0,
  'todayQuantity': 0,
  'adjPrevClose': 447.58,
  'lotsDetails': 'https://api.etrade.com/v1/accounts/SXPtmpM7IZIpymUjhjI-zg/portfolio/364189645900.json',
  'quoteDetails': 'https://api.etrade.com/v1/market/quote/MDB.json',
  'Product': {'symbol': 'MDB',
   'securityType': 'EQ',
   'expiryYear': 0,
   'expiryMonth': 0,
   'expiryDay': 0,
   'strikePrice': 0,
   'productId': {'symbol': 'MDB', 'typeCode': 'EQUITY'}},
  'Quick': {'lastTrade': 444.0,
   'lastTradeTime': 1709317408,
   'change': -3.58,
   'changePct':

In [24]:
data["PortfolioResponse"]["AccountPortfolio"][0]["totalPages"]

1

In [70]:
q_resp = session.get('https://api.etrade.com/v1/market/quote/MDB.json')

In [71]:
q_resp.json()

{'QuoteResponse': {'QuoteData': [{'dateTime': '17:15:49 EST 03-04-2024',
    'dateTimeUTC': 1709590549,
    'quoteStatus': 'CLOSING',
    'ahFlag': 'true',
    'hasMiniOptions': False,
    'All': {'adjustedFlag': False,
     'ask': 440.0,
     'askSize': 500,
     'askTime': '17:15:49 EST 03-04-2024',
     'bid': 435.3,
     'bidExchange': '',
     'bidSize': 2000,
     'bidTime': '17:15:49 EST 03-04-2024',
     'changeClose': 2.16,
     'changeClosePercentage': 0.49,
     'companyName': 'MONGODB INC CL A',
     'daysToExpiration': 0,
     'dirLast': '2',
     'dividend': 0.0,
     'eps': -2.63,
     'estEarnings': 2.924,
     'exDividendDate': 0,
     'high': 443.09,
     'high52': 509.62,
     'lastTrade': 439.0,
     'low': 427.29,
     'low52': 189.5901,
     'open': 437.08,
     'openInterest': 0,
     'optionStyle': '',
     'optionUnderlier': '',
     'previousClose': 436.84,
     'previousDayVolume': 1442620,
     'primaryExchange': 'NSDQ',
     'symbolDescription': 'MONGODB IN

In [30]:
import json

In [31]:
data = json.loads(viewp_resp.text)

In [102]:
viewp_resp.json() == data

True

In [14]:
datetime.fromtimestamp(958743000)

datetime.datetime(2000, 5, 19, 13, 30)

In [34]:
datetime.fromtimestamp(1711522800000/1000)

datetime.datetime(2024, 3, 27, 7, 0)

In [ ]:
data

In [14]:
root_yr = Decimal("1")/Decimal("2.21095890411")

In [18]:
(Decimal("-0.118")+Decimal("1"))**root_yr+Decimal("-1")

Decimal('-0.0552087701011349359455755951')